# Multi-Author Writing Style Analysis

**Authors:** Juan Arturo Abaurrea Calafell, Radu-Andrei Bourceanu

---

## Description

This project is part of the PAN competition tasks, one of the most renowned competitions in the field of Natural Language Processing. Each year, PAN proposes different tasks in a competitive format, providing training datasets where researchers compete to propose solutions.

For this assignment, we selected the **Multi-Author Writing Style Analysis 2025** task from the Authorship track.

### Objectives

1. **Preparation**: Load and preprocess the dataset for the selected task
2. **Training**: Design, justify, and implement a model using at least one fine-tuned Transformer with an original approach
3. **Evaluation**: Evaluate the proposal on test/validation sets and compare results with baseline models and other published approaches

---

## Task Description: Multi-Author Writing Style Analysis

**Dataset:** [Zenodo Record 14891299](https://zenodo.org/records/14891299)

### The Challenge

The goal of the style change detection task is to identify text positions within a given multi-author document at which the author switches.

### Specific Task

For a given text, find all positions of writing style change at the sentence level. Specifically, for each pair of consecutive sentences, assess whether there was a style change (i.e., different authors wrote the sentences).

### Dataset Difficulty Levels

The dataset provides three difficulty levels with carefully controlled topic variation:

| Level | Description | Topic Control |
|-------|-------------|---------------|
| **Easy** | Sentences cover a variety of topics | Approaches can leverage topic information |
| **Medium** | Small topical variety | Must focus more on style |
| **Hard** | All sentences on the same topic | Pure style-based detection required |

### Dataset Structure

Each difficulty level is split into:
- **Training set (70%)**: With ground truth for model development
- **Validation set (15%)**: With ground truth for evaluation and optimization
- **Test set (15%)**: For final evaluation

All documents are in English and may contain an arbitrary number of style changes, but changes only occur between sentences (a single sentence is always authored by a single author).

### Citation

> Zangerle, E., Mayerl, M., Potthast, M., & Stein, B. (2025). PAN25 Multi-Author Writing Style Analysis [Data set]. Zenodo. https://doi.org/10.5281/zenodo.14891299

---

## Our Approach

We implement and compare several transformer-based approaches:

1. **Custom Lightweight Transformer**: A small transformer trained from scratch (~7M parameters)
2. **Pretrained BERT Models**: Fine-tuned HuggingFace models (e.g., bert-mini)
3. **Siamese Architectures**: Models that encode each sentence separately and compare embeddings

To handle the significant class imbalance (most sentences do not switch author), we employ:
- Weighted sampling during training
- Label smoothing for regularization
- Hyperparameter tuning to prevent overfitting

---

## Notebook Structure

- **Part 1: Exploratory Data Analysis (EDA)** - Data loading, preprocessing, and analysis
- **Part 2: Model Architecture** - Definition of encoders, classification heads, and full models
- **Part 3: Training** - Training loop with class imbalance handling
- **Part 4: Single Model Experiment** - Training and evaluating a single model with visualization
- **Part 5: Model Comparison** - Training/loading multiple models and comparing test set performance
- **Part 6: Inference** - Example predictions and interactive testing
- **Conclusion** - Summary of findings and future improvements

---
# Part 1: Setup and Data Loading

## 1.1 Imports and Configuration

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # to avoid potential issues with macOS and some Windows setups
import requests
import zipfile
import pandas as pd
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_SEED = 42

NUM_WORKERS = 0 if os.name == 'nt' else 4  # 0 for Windows, 4 for Linux/Mac
ALWAYS_PARSE = False  # Set to True to always re-parse the data, even if cached files exist
ALWAYS_TRAIN = False  # Set to True to always re-train the model, even if trained models in the results directory exist

## 1.2 Data Download

The dataset comes from the PAN25 Multi-Author Analysis task. We will also store it locally for easy access.

In [ ]:
DATA_URL = "https://zenodo.org/records/14891299/files/pan25-multi-author-analysis.zip"
DATA_DIR = "data"
ZIP_FILE_NAME = "pan25-multi-author-analysis.zip"
ZIP_FILE_PATH = os.path.join(DATA_DIR, ZIP_FILE_NAME)
DIFFICULTY_LEVELS = ['easy', 'medium', 'hard']

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

if not os.path.exists(ZIP_FILE_PATH):
    print("Downloading dataset...")
    response = requests.get(DATA_URL, stream=True)
    response.raise_for_status()

    with open(ZIP_FILE_PATH, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print(f"Downloaded to {ZIP_FILE_PATH}")
else:
    print(f"Dataset already exists at {ZIP_FILE_PATH}")

if not all(level in os.listdir(DATA_DIR) for level in DIFFICULTY_LEVELS):
    print("Extracting dataset...")
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)
    print(f"Extracted to {DATA_DIR}")
else:
    print("Dataset already extracted")

## 1.3 Data Loading Function

The `load_data` function processes the raw problem files and truth labels to create sentence pairs with binary labels:
- **Label = False (0)**: Same author wrote both sentences
- **Label = True (1)**: Different authors wrote the sentences

The function supports:
- Loading specific difficulty levels
- Creating all possible sentence pairs (for data augmentation)
- Swapping sentence order to prevent the model from relying too heavily on the position of sentences

In [ ]:
def load_data(data_dir: str, difficulty_levels: list[str] | str, use_all_possible_pairs: bool = False, swap_same_author_sentences: bool = False, swap_different_author_sentences: bool = False, load_problems: int = None) -> pd.DataFrame:
    """
    Loads data from the specified directory and difficulty levels and returns it as a single dataframe.

    Parameters:
        data_dir: Directory containing the data.
        difficulty_levels: List of difficulty levels to load (e.g., ['easy', 'medium']).
        use_all_possible_pairs: For sentences by the same author (consecutive 0s), creates all possible pairs.
        load_problems: Maximum number of problem files to load per split and difficulty level. If None, loads all.
        swap_same_author_sentences, swap_different_author_sentences: If True, creates additional rows with sentence1 and sentence2 swapped for the respective case.
    Returns:
        pd.DataFrame: DataFrame with columns 'sentence1', 'sentence2', 'label'.
    """
    if not isinstance(difficulty_levels, (list, set, tuple)):
        difficulty_levels = [difficulty_levels]
    
    rows = []
    skipped_count = 0
    
    for split in os.listdir(os.path.join(data_dir, difficulty_levels[0])):
        for level in difficulty_levels:
            print(f"Processing split: {split}, level: {level}")
            split_dir = os.path.join(data_dir, level, split)
            
            if not os.path.exists(split_dir):
                continue
            
            problem_files = sorted([f for f in os.listdir(split_dir) if f.startswith('problem-') and f.endswith('.txt')])
            
            for idx, problem_file in enumerate(problem_files):
                if load_problems is not None and idx >= load_problems:
                    break
                
                problem_num = problem_file.replace('problem-', '').replace('.txt', '')
                truth_file = f'truth-problem-{problem_num}.json'
                
                problem_path = os.path.join(split_dir, problem_file)
                truth_path = os.path.join(split_dir, truth_file)
                
                if not os.path.exists(truth_path):
                    skipped_count += 1
                    continue
                
                with open(problem_path, 'r', encoding='utf-8') as f:
                    sentences = [line.strip() for line in f.readlines() if line.strip()]
                
                with open(truth_path, 'r', encoding='utf-8') as f:
                    truth_data = json.load(f)
                    authors = truth_data.get('authors', 0)
                    changes = truth_data.get('changes', [])
                
                if len(sentences) != len(changes) + 1 or authors < 1:
                    skipped_count += 1
                    continue

                if not use_all_possible_pairs:
                    # Create pairs only between 2 consecutive sentences
                    for i in range(len(sentences) - 1):
                        rows.append({
                            'sentence1': sentences[i],
                            'sentence2': sentences[i + 1],
                            'label': changes[i] != 0
                        })
                else:
                    # Create all possible pairs (more complex logic for multiple authors)
                    if authors == 2:
                        author_sentences = [[], []]
                        current_author = 0
                        
                        for i in range(len(sentences)):
                            author_sentences[current_author].append(sentences[i])
                            if i < len(changes) and changes[i] != 0:
                                current_author = 1 - current_author
                        
                        for author_group in author_sentences:
                            for i in range(len(author_group)):
                                for j in range(i + 1, len(author_group)):
                                    rows.append({
                                        'sentence1': author_group[i],
                                        'sentence2': author_group[j],
                                        'label': False
                                    })
                        
                        for sent_a in author_sentences[0]:
                            for sent_b in author_sentences[1]:
                                rows.append({
                                    'sentence1': sent_a,
                                    'sentence2': sent_b,
                                    'label': True
                                })
                    else:
                        i = 0
                        while i < len(sentences):
                            j = i
                            while j < len(changes) and changes[j] == 0:
                                j += 1
                            
                            group_size = j - i + 1
                            
                            if group_size > 1:
                                for k in range(i, j + 1):
                                    for l in range(k + 1, j + 1):
                                        rows.append({
                                            'sentence1': sentences[k],
                                            'sentence2': sentences[l],
                                            'label': False
                                        })
                            
                            if j < len(changes) and changes[j] != 0:
                                rows.append({
                                    'sentence1': sentences[j],
                                    'sentence2': sentences[j + 1],
                                    'label': True
                                })
                            
                            i = j + 1
    
    df = pd.DataFrame(rows)
    df['sentence1'] = df['sentence1'].astype('string')
    df['sentence2'] = df['sentence2'].astype('string')
    df['label'] = df['label'].astype('boolean')

    print(f"Total documents skipped due to mismatches: {skipped_count}")
    
    if swap_same_author_sentences:
        df_same = df[df['label'] == False].copy()
        df_same['sentence1'], df_same['sentence2'] = df_same['sentence2'], df_same['sentence1']
        rows_added = len(df_same)
        df = pd.concat([df, df_same], ignore_index=True)
        print(f"Added {rows_added} swapped same-author sentence pairs to the dataset")

    if swap_different_author_sentences:
        df_diff = df[df['label'] == True].copy()
        df_diff['sentence1'], df_diff['sentence2'] = df_diff['sentence2'], df_diff['sentence1']
        rows_added = len(df_diff)
        df = pd.concat([df, df_diff], ignore_index=True)
        print(f"Added {rows_added} swapped different-author sentence pairs to the dataset")

    return df

## 1.4 Load and Explore Data

After loading the data, it is cached to CSV to avoid re-parsing on subsequent runs.

In [ ]:
DATA_LOADING_PARAMETERS = {
    "difficulty_levels": DIFFICULTY_LEVELS[0],  # 'easy'
    "swap_different_author_sentences": False,  # True to augment underrepresented class by swapping sentences
    "load_problems": None  # Set to a number to limit data for faster experimentation
}

csv_path = os.path.join(DATA_DIR, 'loaded_data.csv')
if not os.path.exists(csv_path) or ALWAYS_PARSE:
    df = load_data(DATA_DIR, **DATA_LOADING_PARAMETERS)
    df.to_csv(csv_path, index=False)
    print(f"Saved loaded data to {csv_path}\n")
else:
    df = pd.read_csv(csv_path, dtype={'sentence1': 'string', 'sentence2': 'string', 'label': 'boolean'})
    print(f"Loaded data from {csv_path}\n")

## 1.5 Exploratory Data Analysis

Let's examine the dataset characteristics including class distribution, sentence lengths, and outlier removal.

### Class Distribution

In [ ]:
print(f"Total samples: {len(df):,}")
print(f"First 5 rows:")
combined_df = pd.concat([
    df.head(3),
    df.sample(4, random_state=RANDOM_SEED).sort_index(),
    df.tail(3)
])
display(combined_df)

In [ ]:
label_counts = df['label'].value_counts()
print(f"\nLabel value counts:")
print(label_counts)
print(f"\nClass imbalance ratio: {label_counts[False] / label_counts[True]:.2f}:1 (Same:Different authors)")

fig, ax = plt.subplots(figsize=(8, 5))
colors = ['#2ecc71', '#e74c3c']
bars = ax.bar(['Same Author (False)', 'Different Authors (True)'], label_counts.values, color=colors)
ax.set_ylabel('Count')
ax.set_title('Class Distribution')
for bar, count in zip(bars, label_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100, f'{count:,}', ha='center', va='bottom')
plt.tight_layout()
plt.show()

print(f"\nNote: The dataset is imbalanced with {label_counts[False] / label_counts.sum():.2%} same-author pairs.")
print("We address this in different ways during training.")

### Sentence Length Analysis

We'll analyze the distribution of sentence lengths and identify outliers to clean the dataset.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['sentence1'].str.len(), bins=50, alpha=0.7, label='Sentence 1', color='#3498db')
axes[0].hist(df['sentence2'].str.len(), bins=50, alpha=0.7, label='Sentence 2', color='#e74c3c')
axes[0].set_xlabel('Character Length')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Sentence Length Distribution')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].boxplot([df['sentence1'].str.len(), df['sentence2'].str.len()],
                labels=['Sentence 1', 'Sentence 2'],
                patch_artist=True,
                boxprops=dict(facecolor='#3498db', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))
axes[1].set_ylabel('Character Length')
axes[1].set_title('Sentence Length Comparison')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
all_sentences = (
    pd.concat([
        df[['sentence1']].rename(columns={'sentence1': 'sentence'}),
        df[['sentence2']].rename(columns={'sentence2': 'sentence'})
    ])
    .drop_duplicates(subset=['sentence'])
    .assign(length=lambda x: x['sentence'].str.len())
)

def show_sentences(title, ascending):
    print(title)
    display(
        all_sentences
        .sort_values('length', ascending=ascending)
        .head(5)
        .reset_index(drop=True)
    )

show_sentences("Top 5 longest sentences:", ascending=False)
show_sentences("Top 5 shortest sentences:", ascending=True)


### Removing Extreme Cases

We remove extreme outliers for several important reasons:

1. **Data Quality Issues**: Very short sentences (< 10 characters) are often:
   - Single punctuation marks or symbols ("?", "!", ".")
   - Emojis or meaningless fragments
   - Not representative of actual writing style

2. **Noise in Long Sentences**: Very long sentences (> 2000 characters) often contain:
   - Repetitive or copy-pasted text
   - Malformed data from scraping errors
   - Unusual artifacts (e.g., excessive dots or special characters)

3. **Model Training**: 
   - Extreme outliers can dominate loss calculations and gradient updates
   - They don't represent typical sentence pairs in real-world authorship scenarios
   - Can skew the distribution and make training unstable

4. **Statistical Validity**:
   - The remaining distribution is more representative of genuine writing
   - Helps the model learn meaningful stylistic patterns rather than memorizing anomalies

By removing these edge cases, we improve data quality while retaining the vast majority of valid examples.

In [ ]:
df_lengths = (
    pd.concat([
        df[['sentence1']].rename(columns={'sentence1': 'sentence'}),
        df[['sentence2']].rename(columns={'sentence2': 'sentence'})
    ])
    .drop_duplicates()
)
df_lengths['length'] = df_lengths['sentence'].str.len()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.histplot(df_lengths['length'], bins=50, kde=True, ax=axes[0])
axes[0].set_title("Initial Sentence Length Distribution")
axes[0].set_xlabel("Character Length")
axes[0].set_ylabel("Frequency")

print("\nInitial sentence length statistics:")
print(df_lengths['length'].describe(percentiles=[0.01, 0.025, 0.95, 0.99]))

sns.histplot(df_lengths['length'], bins=50, kde=True, ax=axes[1])
axes[1].set_title("Distribution with Cutoffs")
axes[1].axvline(x=df_lengths['length'].quantile(0.025), color='orange', linestyle='--', label='2.5th Percentile')
axes[1].axvline(x=df_lengths['length'].quantile(0.99), color='red', linestyle='--', label='99th Percentile')
axes[1].set_xlabel("Character Length")
axes[1].set_ylabel("Frequency")
axes[1].legend()

# Filter outliers
upper_cutoff = df_lengths['length'].quantile(0.99)
lower_cutoff = df_lengths['length'].quantile(0.025)
print(f"\nRemoving sentences shorter than {lower_cutoff} and longer than {upper_cutoff} characters as outliers.")

df = df[
    (df['sentence1'].str.len() >= lower_cutoff) &
    (df['sentence1'].str.len() <= upper_cutoff) &
    (df['sentence2'].str.len() >= lower_cutoff) &
    (df['sentence2'].str.len() <= upper_cutoff)
].reset_index(drop=True)

# Recalculate lengths after filtering
df_lengths = (
    pd.concat([
        df[['sentence1']].rename(columns={'sentence1': 'sentence'}),
        df[['sentence2']].rename(columns={'sentence2': 'sentence'})
    ])
    .drop_duplicates()
)
df_lengths['length'] = df_lengths['sentence'].str.len()

sns.histplot(df_lengths['length'], bins=50, kde=True, ax=axes[2])
axes[2].set_title("After Outlier Removal")
axes[2].set_xlabel("Character Length")
axes[2].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

label_counts = df['label'].value_counts()
print(f"\nAfter this removal, the dataset is still imbalanced with {label_counts[False] / label_counts.sum():.2%} same-author pairs.")

After removing outliers (sentences below 2.5th percentile and above 99th percentile), even though we have made the underrepresented class less common, we have a cleaner distribution for model training.

### Key Statistics After Cleaning

In [ ]:
# Shortest individual sentence
shortest_len = min(df['sentence1'].str.len().min(), df['sentence2'].str.len().min())
if df['sentence1'].str.len().min() <= df['sentence2'].str.len().min():
    shortest_sentence = df.loc[df['sentence1'].str.len().idxmin(), 'sentence1']
    shortest_type = "sentence1"
else:
    shortest_sentence = df.loc[df['sentence2'].str.len().idxmin(), 'sentence2']
    shortest_type = "sentence2"

print(f"Shortest sentence length: {shortest_len}")
print(f"Shortest sentence ({shortest_type}): {shortest_sentence}\n")

# Longest individual sentence
longest_sent1 = df['sentence1'].str.len().max()
longest_sent2 = df['sentence2'].str.len().max()
longest_len = max(longest_sent1, longest_sent2)
if longest_sent1 >= longest_sent2:
    longest_sentence = df.loc[df['sentence1'].str.len().idxmax(), 'sentence1']
    longest_type = "sentence1"
else:
    longest_sentence = df.loc[df['sentence2'].str.len().idxmax(), 'sentence2']
    longest_type = "sentence2"

print(f"Longest sentence length: {longest_len}")
print(f"Longest sentence ({longest_type}): {longest_sentence}\n")

# Longest combined sentence pair (sum of both sentences in the same row)
df['combined_len'] = df['sentence1'].str.len() + df['sentence2'].str.len()
longest_combined_idx = df['combined_len'].idxmax()
longest_combined = df.loc[longest_combined_idx, 'combined_len']
longest_combined_sent1 = df.loc[longest_combined_idx, 'sentence1']
longest_combined_sent2 = df.loc[longest_combined_idx, 'sentence2']

print(f"Longest combined pair length: {longest_combined}")
print(f"Sentence 1: {longest_combined_sent1}")
print(f"Sentence 2: {longest_combined_sent2}")

print(f"Mean sentence length: {df['combined_len'].mean():.1f} characters")

# Clean up temporary column
df.drop('combined_len', axis=1, inplace=True)


## 1.6 Tokenization Analysis

Before building our models, we need to determine an appropriate `max_length` for our tokenizer. This involves analyzing the distribution of token lengths in our dataset to balance between:
- **Capturing most of the content**: Not truncating too many sentences
- **Computational efficiency**: Shorter sequences = faster training
- **Memory constraints**: Longer sequences use more GPU memory and may cause more padding to be used, which is inefficient

In [ ]:
# We will use GPT-2 tokenizer for analysis as a representative tokenizer since it is the une we used in our custom model.
analysis_tokenizer = AutoTokenizer.from_pretrained('gpt2')
analysis_tokenizer.pad_token = analysis_tokenizer.eos_token

def get_token_length(text: str, tokenizer) -> int:
    """Get the number of tokens for a text string."""
    return len(tokenizer.encode(text, add_special_tokens=True))

print("Analyzing token lengths...")
df['sent1_tokens'] = df['sentence1'].apply(lambda x: get_token_length(x, analysis_tokenizer))
df['sent2_tokens'] = df['sentence2'].apply(lambda x: get_token_length(x, analysis_tokenizer))

df['combined_tokens'] = df.apply(
    lambda row: len(analysis_tokenizer.encode(
        row['sentence1'], row['sentence2'], add_special_tokens=True
    )), axis=1
)

df['max_single_tokens'] = df[['sent1_tokens', 'sent2_tokens']].max(axis=1)

print("\nStandard Mode (concatenated sentences):")
print(df['combined_tokens'].describe())

print("\nSiamese Mode (individual sentences):")
print(df['max_single_tokens'].describe())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['combined_tokens'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df['combined_tokens'].quantile(0.95), color='red', linestyle='--', 
                label=f"95th percentile: {df['combined_tokens'].quantile(0.95):.0f}")
axes[0].axvline(df['combined_tokens'].quantile(0.99), color='orange', linestyle='--',
                label=f"99th percentile: {df['combined_tokens'].quantile(0.99):.0f}")
axes[0].set_xlabel('Token Length (Combined)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Standard Mode: Concatenated Token Lengths')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Siamese mode distribution
axes[1].hist(df['max_single_tokens'], bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(df['max_single_tokens'].quantile(0.95), color='red', linestyle='--',
                label=f"95th percentile: {df['max_single_tokens'].quantile(0.95):.0f}")
axes[1].axvline(df['max_single_tokens'].quantile(0.99), color='orange', linestyle='--',
                label=f"99th percentile: {df['max_single_tokens'].quantile(0.99):.0f}")
axes[1].set_xlabel('Token Length (Max Single)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Siamese Mode: Individual Sentence Token Lengths')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

percentiles = [0.90, 0.95, 0.99, 1.0]
print("\nStandard Mode (concatenated):")
for p in percentiles:
    val = df['combined_tokens'].quantile(p)
    pct = p * 100
    truncated = (df['combined_tokens'] > val).sum()
    print(f"  {pct:5.1f}th percentile: {val:6.0f} tokens ({truncated:,} samples would be truncated, {truncated/len(df)*100:.1f}%)")

print("\nSiamese Mode (individual):")
for p in percentiles:
    val = df['max_single_tokens'].quantile(p)
    pct = p * 100
    truncated = (df['max_single_tokens'] > val).sum()
    print(f"  {pct:5.1f}th percentile: {val:6.0f} tokens ({truncated:,} samples would be truncated, {truncated/len(df)*100:.1f}%)")

print("\nWe will use max_length=128 as it captures 99%+ of sequences (only ~1% truncated) while maximizing efficiency.")

# Clean up temporary columns
df = df.drop(columns=['sent1_tokens', 'sent2_tokens', 'combined_tokens', 'max_single_tokens'])

---
# Part 2: Model Architecture

## 2.1 Device Setup

We check for GPU availability to accelerate training.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2.2 Model Constants

In [ ]:
CUSTOM_MODEL_NAME = 'custom-lightweight-transformer'  # Name for the custom model
SIAMESE_PREFIX = 'siamese-'  # Prefix for siamese model variants

## 2.3 Encoder Architectures

We define a base encoder class and two implementations:

1. **LightweightTransformerEncoder**: A custom, small transformer trained from scratch
2. **PretrainedEncoder**: A wrapper around HuggingFace pretrained models

Both encoders output a fixed-size embedding for each input sequence.

In [ ]:
class BaseEncoder(nn.Module, ABC):
    """Abstract base class for all encoders (custom and pretrained)"""
    
    @abstractmethod
    def encode(self, input_ids, attention_mask=None):
        """Encode input and return pooled representation"""
        pass
    
    @abstractmethod
    def get_output_dim(self) -> int:
        """Return the dimension of the encoder output"""
        pass
    
    @abstractmethod
    def save_pretrained(self, path: str):
        """Save encoder to disk"""
        pass
    
    @classmethod
    @abstractmethod
    def from_pretrained(cls, path: str):
        """Load encoder from disk"""
        pass

In [ ]:
class LightweightTransformerEncoder(BaseEncoder):
    """
    Custom lightweight transformer encoder.
    
    A small transformer that can be trained from scratch.
    Uses learned positional embeddings and returns CLS token representation.
    """
    
    def __init__(self, vocab_size, d_model, nhead, num_layers,
                 dim_feedforward, max_length, dropout, pad_token_id):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_token_id)
        self.position_embedding = nn.Embedding(max_length, d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=dropout, activation='gelu', batch_first=True, norm_first=True
        )
        
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers, enable_nested_tensor=False)
        
        self.d_model = d_model
        self.config = {
            'vocab_size': vocab_size,
            'd_model': d_model,
            'nhead': nhead,
            'num_layers': num_layers,
            'dim_feedforward': dim_feedforward,
            'max_length': max_length,
            'dropout': dropout,
            'pad_token_id': pad_token_id
        }
    
    def encode(self, input_ids, attention_mask=None):
        seq_len = min(input_ids.size(1), self.config['max_length'])
        pos_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        x = self.embedding(input_ids[:, :seq_len]) + self.position_embedding(pos_ids)

        if attention_mask is not None:
            attention_mask = attention_mask[:, :seq_len]
        
        src_key_padding_mask = (attention_mask == 0) if attention_mask is not None else None
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        
        return x[:, 0, :]  # CLS token (first position)
    
    def get_output_dim(self) -> int:
        return self.d_model
    
    def save_pretrained(self, path: str):
        os.makedirs(path, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(path, 'pytorch_model.bin'))
        with open(os.path.join(path, 'config.json'), 'w') as f:
            json.dump(self.config, f, indent=2)
    
    @classmethod
    def from_pretrained(cls, path: str):
        with open(os.path.join(path, 'config.json'), 'r') as f:
            config = json.load(f)
        model = cls(**config)
        model.load_state_dict(torch.load(os.path.join(path, 'pytorch_model.bin'), weights_only=True))
        return model
    
    def num_parameters(self) -> int:
        return sum(p.numel() for p in self.parameters())

In [ ]:
class PretrainedEncoder(BaseEncoder):
    """
    Wrapper for HuggingFace pretrained models.
    
    Leverages pretrained weights from models like BERT, RoBERTa, etc.
    Returns either pooler_output or CLS token embedding.
    """
    
    def __init__(self, model_name: str):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.model_name = model_name
        self._output_dim = self.model.config.hidden_size
    
    def encode(self, input_ids, attention_mask=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
            return outputs.pooler_output
        return outputs.last_hidden_state[:, 0, :]  # CLS token
    
    def get_output_dim(self) -> int:
        return self._output_dim
    
    def save_pretrained(self, path: str):
        self.model.save_pretrained(path)
        with open(os.path.join(path, 'encoder_info.json'), 'w') as f:
            json.dump({'model_name': self.model_name}, f)
    
    @classmethod
    def from_pretrained(cls, path: str):
        instance = object.__new__(cls)
        nn.Module.__init__(instance)
        instance.model = AutoModel.from_pretrained(path)
        instance._output_dim = instance.model.config.hidden_size
        with open(os.path.join(path, 'encoder_info.json'), 'r') as f:
            info = json.load(f)
        instance.model_name = info['model_name']
        return instance
    
    def num_parameters(self) -> int:
        return sum(p.numel() for p in self.parameters())

## 2.4 Classification Head

A simple MLP classification head that takes encoder output and produces class logits.

In [ ]:
class ClassificationHead(nn.Module):
    """Simple MLP classification head with dropout regularization."""
    
    def __init__(self, input_dim: int, dropout: float, num_labels: int = 2):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(input_dim // 2, num_labels)
        )
        self.config = {'input_dim': input_dim, 'num_labels': num_labels, 'dropout': dropout}
    
    def forward(self, x):
        return self.classifier(x)
    
    def save_pretrained(self, path: str):
        torch.save(self.state_dict(), os.path.join(path, 'classifier.bin'))
        with open(os.path.join(path, 'classifier_config.json'), 'w') as f:
            json.dump(self.config, f, indent=2)
    
    @classmethod
    def from_pretrained(cls, path: str):
        with open(os.path.join(path, 'classifier_config.json'), 'r') as f:
            config = json.load(f)
        model = cls(**config)
        model.load_state_dict(torch.load(os.path.join(path, 'classifier.bin'), weights_only=True))
        return model

## 2.5 Full Model Architectures

### AuthorshipModel

This model supports two modes:

1. **Standard mode**: Takes concatenated sentence pairs as input (using tokenizer's SEP token)
2. **Siamese mode**: Encodes each sentence separately and combines embeddings using configurable similarity methods:
   - **concat**: Concatenation of both embeddings `[emb1, emb2]`
   - **diff**: Absolute difference `|emb1 - emb2|` (captures element-wise dissimilarity)
   - **mult**: Element-wise multiplication `emb1 * emb2` (captures feature interaction)
   - **cos**: Cosine similarity (captures global angular alignment)

The similarity methods can be combined in any configuration by passing a list (e.g., `['concat', 'diff', 'mult', 'cos']` for all features, or `['cos']` for just cosine similarity).

In [ ]:
class AuthorshipModel(nn.Module):
    """
    Model for authorship verification.
    
    Can operate in standard mode (concatenated input) or siamese mode (separate encoding).
    In siamese mode, embeddings are combined using the specified similarity methods.
    """
    
    def __init__(self, encoder: BaseEncoder, dropout: float, num_labels: int = 2,
                 siamese: bool = False, similarity_methods: list = ['concat', 'diff', 'mult', 'cos']):
        super().__init__()
        
        self.encoder = encoder
        self.siamese = siamese
        self.similarity_methods = similarity_methods if siamese else []  # Clear for non-siamese
        self.num_labels = num_labels
        
        encoder_dim = encoder.get_output_dim()
        
        if siamese:
            classifier_input = 0
            
            if 'concat' in self.similarity_methods:
                classifier_input += encoder_dim * 2
            if 'diff' in self.similarity_methods:
                classifier_input += encoder_dim
            if 'mult' in self.similarity_methods:
                classifier_input += encoder_dim
            if 'cos' in self.similarity_methods:
                classifier_input += 1
            
            if classifier_input == 0:
                raise ValueError("At least one similarity method must be specified for siamese mode")
        else:
            classifier_input = encoder_dim
        
        self.classifier = ClassificationHead(classifier_input, num_labels=num_labels, dropout=dropout)
        
        self.config = {
            'num_labels': num_labels,
            'dropout': dropout,
            'siamese': siamese,
            'similarity_methods': self.similarity_methods
        }
    
    def forward(self, input_ids1, attention_mask1, input_ids2=None, attention_mask2=None, 
                labels=None, class_weights=None, label_smoothing=0.0):
        """
        Forward pass.
        
        Args:
            input_ids1, attention_mask1: Primary input (or concatenated sentences in standard mode)
            input_ids2, attention_mask2: Second sentence (siamese mode only)
            labels: Ground truth labels for loss computation
            class_weights: Optional weights for imbalanced classes
            label_smoothing: Label smoothing factor for regularization
        """
        if self.siamese:
            if input_ids2 is None or attention_mask2 is None:
                raise ValueError("Siamese mode requires input_ids2 and attention_mask2")
            
            emb1 = self.encoder.encode(input_ids1, attention_mask1)
            emb2 = self.encoder.encode(input_ids2, attention_mask2)
            
            features = []
            
            if 'concat' in self.similarity_methods:
                features.extend([emb1, emb2])
            
            if 'diff' in self.similarity_methods:
                diff = torch.abs(emb1 - emb2)
                features.append(diff)
            
            if 'mult' in self.similarity_methods:
                mult = emb1 * emb2
                features.append(mult)
            
            if 'cos' in self.similarity_methods:
                cos_sim = F.cosine_similarity(emb1, emb2, dim=-1).unsqueeze(-1)
                features.append(cos_sim)
            
            combined = torch.cat(features, dim=-1)
            logits = self.classifier(combined)
        else:
            emb = self.encoder.encode(input_ids1, attention_mask1)
            logits = self.classifier(emb)
        
        output = type('Output', (), {'logits': logits})()
        
        if labels is not None:
            output.loss = F.cross_entropy(logits, labels, weight=class_weights, label_smoothing=label_smoothing)
        
        return output
    
    def save_pretrained(self, path: str):
        os.makedirs(path, exist_ok=True)
        self.encoder.save_pretrained(os.path.join(path, 'encoder'))
        self.classifier.save_pretrained(path)
        with open(os.path.join(path, 'model_config.json'), 'w') as f:
            json.dump(self.config, f, indent=2)
        print(f"Model saved to {path}")
    
    @classmethod
    def from_pretrained(cls, path: str, encoder_class: BaseEncoder):
        with open(os.path.join(path, 'model_config.json'), 'r') as f:
            config = json.load(f)
        
        encoder = encoder_class.from_pretrained(os.path.join(path, 'encoder'))
        model = cls(encoder, **config)
        model.classifier = ClassificationHead.from_pretrained(path)
        return model
    
    def num_parameters(self) -> int:
        return sum(p.numel() for p in self.parameters())

## 2.6 Dataset Classes

We define two dataset classes:

1. **StandardDataset**: Tokenizes sentence pairs together
2. **SiameseDataset**: Tokenizes each sentence separately

In [ ]:
class StandardDataset(Dataset):
    """Dataset that tokenizes sentence pairs together (for non-siamese models)."""
    
    def __init__(self, dataframe, tokenizer, max_length):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        encoding = self.tokenizer(
            row['sentence1'], row['sentence2'],
            padding='max_length', truncation=True,
            max_length=self.max_length, return_tensors='pt'
        )
        return (
            encoding['input_ids'].squeeze(0),
            encoding['attention_mask'].squeeze(0),
            torch.tensor(int(row['label']), dtype=torch.long)
        )


class SiameseDataset(Dataset):
    """Dataset that tokenizes each sentence separately (for siamese models)."""
    
    def __init__(self, dataframe, tokenizer, max_length):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc1 = self.tokenizer(row['sentence1'], padding='max_length', truncation=True,
                              max_length=self.max_length, return_tensors='pt')
        enc2 = self.tokenizer(row['sentence2'], padding='max_length', truncation=True,
                              max_length=self.max_length, return_tensors='pt')
        return (
            enc1['input_ids'].squeeze(0), enc1['attention_mask'].squeeze(0),
            enc2['input_ids'].squeeze(0), enc2['attention_mask'].squeeze(0),
            torch.tensor(int(row['label']), dtype=torch.long)
        )

## 2.7 Model Configuration

The `ModelConfig` dataclass and `get_model_config` function provide an interface for creating and configuring different model types.

In [ ]:
def get_model_size_mb(model: nn.Module, assume_uniform_dtype: bool = True) -> float:
    """Calculate actual model size in MB based on parameter dtypes."""
    params = list(model.parameters())
    if not params:
        return 0.0
    
    if assume_uniform_dtype:
        return (model.num_parameters() * params[0].element_size()) / 1e6
    
    total_bytes = 0
    for param in model.parameters():
        total_bytes += param.numel() * param.element_size()
    return total_bytes / 1e6

@dataclass
class ModelConfig:
    """Configuration container for a model."""
    model_name: str
    model: AuthorshipModel
    device: torch.device
    tokenizer: AutoTokenizer
    max_length: int
    batch_size: int
    learning_rate: float
    num_epochs: int
    model_path: str
    is_siamese: bool
    is_custom: bool
    dataset_class: type  # Either StandardDataset or SiameseDataset

def get_model_config(
    model_name: str,
    device: torch.device,
    max_length: int = 128,  # Value chosen based on previous analysis
    num_labels: int = 2,
    # Custom model parameters
    custom_d_model: int = 128,
    custom_nhead: int = 4,
    custom_num_layers: int = 2,
    custom_dim_feedforward: int = 256,
    dropout: float = 1/3,  # 0,33... dropout
    # Siamese parameters
    siamese_similarity_methods: list[str] = ['concat', 'diff', 'mult', 'cos'],
    # Training parameters
    batch_size: int = None,
    learning_rate: float = None,
    num_epochs: int = None
) -> ModelConfig:
    """
    Creates model configuration based on model name. For custom models, it uses gpt2's tokenizer.
    
    Supported formats:
        - 'custom-lightweight-transformer': Custom transformer
        - 'siamese-custom-lightweight-transformer': Siamese custom transformer  
    It also supports pretrained HuggingFace models with and without 'siamese-' prefix:
        - 'prajjwal1/bert-mini'
        - 'microsoft/deberta-v3-small'
        - 'Qwen/Qwen2.5-0.5B'
    """
    if (not batch_size) or (not learning_rate) or (not num_epochs):
        print("WARNING: One or more training parameters (batch_size, learning_rate, num_epochs) were not provided in the get_model_config function and will be set to default values.")
    
    if model_name.endswith(CUSTOM_MODEL_NAME):
        if not batch_size:
            batch_size = 64
        if not learning_rate:
            learning_rate = 1e-3
        if not num_epochs:
            num_epochs = 20  # 10 may be enough
    else:
        if not batch_size:
            batch_size = 16
        if not learning_rate:
            learning_rate = 2e-5  # maybe even 1e-5
        if not num_epochs:
            num_epochs = 10  # 5 may be enough
    is_siamese = model_name.startswith(SIAMESE_PREFIX)
    base_model_name = model_name[len(SIAMESE_PREFIX):] if is_siamese else model_name
    is_custom = base_model_name == CUSTOM_MODEL_NAME
    
    model_path = os.path.join('.', 'results', model_name.replace("/", "-"))
    
    print("=" * 60)
    print(f"Model: {model_name}")
    print(f"Type: {'Siamese ' if is_siamese else ''}{'Custom' if is_custom else 'Pretrained'}")
    print("=" * 60)
    
    if is_custom:
        tokenizer = AutoTokenizer.from_pretrained('gpt2')
        tokenizer.pad_token = tokenizer.eos_token
        encoder = LightweightTransformerEncoder(
            vocab_size=len(tokenizer),
            d_model=custom_d_model,
            nhead=custom_nhead,
            num_layers=custom_num_layers,
            dim_feedforward=custom_dim_feedforward,
            max_length=max_length,
            dropout=dropout,
            pad_token_id=tokenizer.pad_token_id
        )
    else:
        tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        encoder = PretrainedEncoder(base_model_name)

    model = AuthorshipModel(
        encoder=encoder,
        num_labels=num_labels,
        dropout=dropout,
        siamese=is_siamese,
        similarity_methods=siamese_similarity_methods
    ).to(device)
    
    dataset_class = SiameseDataset if is_siamese else StandardDataset
    
    print(f"Parameters: {model.num_parameters():,}")
    print(f"Size: ~{get_model_size_mb(model):.2f} MB")
    
    model.encoder
    return ModelConfig(
        model_name=model_name,
        model=model,
        device=device,
        tokenizer=tokenizer,
        max_length=max_length,
        batch_size=batch_size,
        learning_rate=learning_rate,
        num_epochs=num_epochs,
        model_path=model_path,
        is_siamese=is_siamese,
        is_custom=is_custom,
        dataset_class=dataset_class
    )

def load_model_from_config(config: ModelConfig, move_to_device: bool = True) -> nn.Module:
    """Load a model from disk using its config."""
    path = config.model_path
    
    model = AuthorshipModel.from_pretrained(path, type(config.model.encoder))
    
    if move_to_device:
        model = model.to(config.device)
    return model

## 2.8 Architecture Overview

The modular design we applied allows for flexible experimentation with different encoder types (custom vs. pretrained) and training modes (standard vs. siamese).

The following cell should be rendered as a class diagram of the model architecture. If it does not, you can visualize it here: [https://mermaid.live/](https://mermaid.live/)

If you are using Visual Studio Code, after installing [Markdown Preview Mermaid Support](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) you should be able to visualize it.


```mermaid
classDiagram
    class BaseEncoder {
        <<abstract>>
        +encode(input_ids, attention_mask)*
        +get_output_dim()* int
        +save_pretrained(path)*
        +from_pretrained(path)*
    }
    
    class LightweightTransformerEncoder {
        -embedding: nn.Embedding
        -position_embedding: nn.Embedding
        -transformer: nn.TransformerEncoder
        -d_model: int
        -config: dict
        +encode(input_ids, attention_mask)
        +get_output_dim() int
        +save_pretrained(path)
        +from_pretrained(path)
        +num_parameters() int
    }
    
    class PretrainedEncoder {
        -model: AutoModel
        -model_name: str
        -_output_dim: int
        +encode(input_ids, attention_mask)
        +get_output_dim() int
        +save_pretrained(path)
        +from_pretrained(path)
        +num_parameters() int
    }
    
    class ClassificationHead {
        -classifier: nn.Sequential
        -config: dict
        +forward(x)
        +save_pretrained(path)
        +from_pretrained(path)
    }
    
    class AuthorshipModel {
        -encoder: BaseEncoder
        -classifier: ClassificationHead
        -siamese: bool
        -similarity_methods: list[str]
        -num_labels: int
        -config: dict
        +forward(input_ids1, attention_mask1, ...)
        +save_pretrained(path)
        +from_pretrained(path, encoder_class)
        +num_parameters() int
    }
    
    class ModelConfig {
        <<dataclass>>
        +model_name: str
        +model: AuthorshipModel
        +device: torch.device
        +tokenizer: AutoTokenizer
        +max_length: int
        +batch_size: int
        +learning_rate: float
        +num_epochs: int
        +model_path: str
        +is_siamese: bool
        +is_custom: bool
        +dataset_class: type
    }
    
    class Dataset {
        <<abstract>>
        +__len__()*
        +__getitem__(idx)*
    }
    
    class StandardDataset {
        -df: DataFrame
        -tokenizer: AutoTokenizer
        -max_length: int
        +__len__() int
        +__getitem__(idx) tuple[Tensor, Tensor, Tensor]
    }
    
    class SiameseDataset {
        -df: DataFrame
        -tokenizer: AutoTokenizer
        -max_length: int
        +__len__() int
        +__getitem__(idx) tuple[Tensor, Tensor, Tensor, Tensor, Tensor]
    }
    
    class TrainingResult {
        <<dataclass>>
        +best_f1: float
        +best_epoch: int
        +final_epoch: int
        +training_history: list
    }
    
    BaseEncoder <|-- LightweightTransformerEncoder : implements
    BaseEncoder <|-- PretrainedEncoder : implements
    
    AuthorshipModel *-- BaseEncoder : contains
    AuthorshipModel *-- ClassificationHead : contains

    AuthorshipModel ..> TrainingResult : training produces
    Dataset ..> TrainingResult : used to generate

    Dataset <|-- StandardDataset : implements
    Dataset <|-- SiameseDataset : implements

    ModelConfig --> AuthorshipModel : references
    ModelConfig --> Dataset : references via dataset_class
```

---
# Part 3: Training

## 3.1 Evaluation Metrics

We use several metrics to evaluate model performance:

- **Accuracy**: Overall correctness
- **Precision**: Of predicted positives, how many are correct
- **Recall**: Of actual positives, how many were detected
- **F1 Score**: Harmonic mean of precision and recall (our primary metric)
- **AUC-ROC**: Area under the ROC curve

In [ ]:
def compute_metrics(all_preds, all_labels, all_probs):
    """Compute classification metrics."""
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='binary', zero_division=0
    )
    
    try:
        auc_roc = roc_auc_score(all_labels, all_probs)
    except (ValueError, AttributeError) as e:
        print(f"Warning: Could not compute AUC-ROC ({type(e).__name__}): {e}")
        auc_roc = 0.0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc_roc': auc_roc
    }

def evaluate(model: AuthorshipModel, dataloader: DataLoader, device: torch.device):
    """Evaluate model on a dataset."""
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            batch = [b.to(device) for b in batch]
            
            if model.siamese:
                input_ids1, attention_mask1, input_ids2, attention_mask2, labels = batch
                outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2, labels=labels)
            else:
                input_ids, attention_mask, labels = batch
                outputs = model(input_ids, attention_mask, labels=labels)
            
            total_loss += outputs.loss.item()
            
            probs = F.softmax(outputs.logits, dim=-1)
            preds = torch.argmax(outputs.logits, dim=-1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
    
    metrics = compute_metrics(all_preds, all_labels, all_probs)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics

## 3.2 Training Function

The training function includes:

- **Weighted sampling** (default: enabled) to handle class imbalance by oversampling minority class examples during batch creation
- **Weighted loss** (default: disabled) to handle class imabalance by assigning higher penalty to misclassifications of the underrepresented class
- **Label smoothing** (default: 0.1) for regularization
- **OneCycleLR scheduler** for learning rate scheduling
- **Encoder freezing** (default: disabled) to support progressive fine-tuning:
  - `False`: train all parameters from the start (full fine-tuning)
  - `True`: freeze encoder completely (feature extraction mode)
  - `float` (0-1): freeze encoder for that fraction of epochs, then unfreeze (progressive fine-tuning)
- **Early stopping** to prevent overfitting
- **Gradient clipping** for training stability

Weighted loss is disabled by default since weighted sampling already addresses class imbalance, and combining both techniques can over-correct and hurt performance on the majority class.

In [ ]:
@dataclass
class TrainingResult:
    """Results from training a model."""
    best_f1: float
    best_epoch: int
    final_epoch: int
    training_history: list = field(default_factory=list)

In [ ]:
def train_model(
    config: ModelConfig,
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    max_patience: int = 3,
    weight_decay: float = 0.1,
    label_smoothing: float = 0.1,
    weighted_sampler: bool = True,
    weighted_loss: bool = False,
    freeze_encoder: bool | float = False
) -> tuple[nn.Module, TrainingResult]:
    """
    Train a model with early stopping and various regularization techniques.
    
    Parameters:
        config: ModelConfig object containing model configuration
        train_df, val_df: DataFrames with 'sentence1', 'sentence2', 'label' columns
        max_patience: Early stopping patience (epochs without improvement)
        weight_decay: L2 regularization strength
        label_smoothing: Label smoothing factor for loss
        weighted_sampler: Whether to use weighted random sampler for imbalanced data
        weighted_loss: Whether to use class-weighted loss function
        freeze_encoder: If True, freeze encoder weights; if float, initially freeze for that fraction of epochs
    Returns:
        tuple[nn.Module, TrainingResult]: Best model and training results
    """
    
    PCT_STEPS_FOR_WARMUP = 0.1  # During the first 10% of steps, LR increases from 0 to max_lr
    
    # Epoch in which to unfreeze encoder
    unfreeze_at_epoch = 0
    if isinstance(freeze_encoder, float):
        if not (0.0 < freeze_encoder < 1.0):
            raise ValueError("If freeze_encoder is a float, it must be between 0.0 and 1.0")
        if freeze_encoder < PCT_STEPS_FOR_WARMUP:
            print(f"WARNING: freeze_encoder fraction {freeze_encoder} is less than the warm-up percentage {PCT_STEPS_FOR_WARMUP}. This may lead to suboptimal training since the learning rate will not reach its maximum peak.")
        unfreeze_at_epoch = int(config.num_epochs * freeze_encoder)
        print(f"Encoder frozen until epoch {unfreeze_at_epoch + 1}/{config.num_epochs}")
    elif freeze_encoder is True:
        unfreeze_at_epoch = float('inf')  # never unfreeze
        print(f"Encoder frozen for all epochs")

    model = config.model
    best_model = model
    tokenizer = config.tokenizer
    
    # Compute class weights for imbalanced data
    labels = train_df['label'].astype(int).values
    class_counts = np.bincount(labels)

    if weighted_loss:
        class_weights = torch.tensor(len(labels) / (2 * class_counts), dtype=torch.float32).to(config.device)
    else:
        class_weights = None
    
    # Create datasets
    train_dataset = config.dataset_class(train_df, tokenizer, config.max_length)
    val_dataset = config.dataset_class(val_df, tokenizer, config.max_length)

    # Create data loaders with optional weighted sampling
    if weighted_sampler:
        sample_weights = 1.0 / class_counts[labels]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, num_workers=NUM_WORKERS)
    else:
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=NUM_WORKERS)
    
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=NUM_WORKERS)
    
    # Handle encoder freezing
    if freeze_encoder:
        for param in model.encoder.parameters():
            param.requires_grad = False

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate, weight_decay=weight_decay)
    
    total_steps = len(train_loader) * config.num_epochs
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=config.learning_rate,  # Peak learning rate
        total_steps=total_steps,
        pct_start=PCT_STEPS_FOR_WARMUP,  # Warm-up for first 10% of steps, then decay
        anneal_strategy='cos'  # Softer function compared to linear
    )
    
    print(f"\nTraining {config.model_name}")
    print(f"Train: {len(train_df):,}, Val: {len(val_df):,}")
    print(f"Batch size: {config.batch_size}, LR: {config.learning_rate}, Epochs: {config.num_epochs}")
    print(f"Total steps: {total_steps:,}")
    print(f"Weighted sampler: {weighted_sampler}, Weighted loss: {weighted_loss}, Label smoothing: {label_smoothing}")
    print("-" * 50)

    # Save initial model and tokenizer so that even if training is unsuccessful, we have something saved
    os.makedirs(config.model_path, exist_ok=True)
    model.save_pretrained(config.model_path)
    tokenizer.save_pretrained(config.model_path)
    
    # Training loop
    best_f1 = 0
    best_epoch = 0
    patience_counter = 0
    training_history = []

    for epoch in range(config.num_epochs):
        # if the encoder was frozen, unfreeze it after the specified number of epochs
        if epoch == unfreeze_at_epoch:
            print(f"\nUnfreezing encoder at epoch {epoch+1}/{config.num_epochs}")
            for param in model.encoder.parameters():
                param.requires_grad = True
            
            current_lr = optimizer.param_groups[0]['lr']
    
            optimizer = torch.optim.AdamW(
                filter(lambda p: p.requires_grad, model.parameters()),
                lr=current_lr,  # make sure to continue from current LR so that there is no spike in LR
                weight_decay=weight_decay
            )
            
            remaining_steps = len(train_loader) * (config.num_epochs - epoch)
            # now only using cosine annealing for the remaining steps to smoothly decay the LR (no increase)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=remaining_steps,
                eta_min=current_lr * 0.1  # Minimum = 10% of current LR
            )
        model.train()
        train_loss = 0
        train_preds, train_labels_list = [], []
        
        for batch_idx, batch in enumerate(train_loader):
            batch = [b.to(config.device) for b in batch]
            
            if config.is_siamese:
                input_ids1, attention_mask1, input_ids2, attention_mask2, labels = batch
                outputs = model(input_ids1, attention_mask1, input_ids2, attention_mask2, 
                               labels=labels, class_weights=class_weights, label_smoothing=label_smoothing)
            else:
                input_ids, attention_mask, labels = batch
                outputs = model(input_ids, attention_mask, 
                               labels=labels, class_weights=class_weights, label_smoothing=label_smoothing)
            
            optimizer.zero_grad()
            outputs.loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            
            train_loss += outputs.loss.item()
            preds = torch.argmax(outputs.logits, dim=-1)
            train_preds.extend(preds.cpu().numpy())
            train_labels_list.extend(labels.cpu().numpy())
            
            if (batch_idx + 1) % 50 == 0:
                print(f"  Epoch {epoch+1} - Batch {batch_idx+1}/{len(train_loader)} - Loss: {outputs.loss.item():.4f}")
        
        train_acc = accuracy_score(train_labels_list, train_preds)
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation
        val_metrics = evaluate(model, val_loader, config.device)
        
        epoch_record = {
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'train_acc': train_acc,
            **{f'val_{k}': v for k, v in val_metrics.items()}
        }
        training_history.append(epoch_record)
        
        print(f"Epoch {epoch+1}/{config.num_epochs} stats")
        print(f"  Train - Loss: {avg_train_loss:.4f}, Acc: {train_acc:.4f}")
        print(f"  Val   - Loss: {val_metrics['loss']:.4f}, Acc: {val_metrics['accuracy']:.4f}, "
              f"P: {val_metrics['precision']:.4f}, R: {val_metrics['recall']:.4f}, "
              f"F1: {val_metrics['f1']:.4f}, AUC: {val_metrics['auc_roc']:.4f}")
        
        # Early stopping check
        if val_metrics['f1'] > best_f1:
            best_f1 = val_metrics['f1']
            best_epoch = epoch + 1
            patience_counter = 0
            
            model.save_pretrained(config.model_path)
            best_model = model
            print(f"  New best model saved (F1: {best_f1:.4f})")
        else:
            patience_counter += 1
            if patience_counter >= max_patience:
                print(f"\nEarly stopping at epoch {epoch+1}")
                break
        print()
    
    print("\n" + "=" * 50)
    print(f"Training complete! Best F1: {best_f1:.4f} at epoch {best_epoch}")
    print(f"Best model saved to: {config.model_path}")
    print("=" * 50)
    
    return (
        best_model,
        TrainingResult(
            best_f1=best_f1,
            best_epoch=best_epoch,
            final_epoch=epoch + 1,
            training_history=training_history
        )
    )

## 3.3 Model Comparison Function

This function trains and evaluates multiple models, producing a comparison table.

In [ ]:
def compare_models(
    model_names: list[str],
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    model_config_kwargs: dict = None,
    train_kwargs: dict = None
) -> pd.DataFrame:
    """
    Train and compare multiple models.
    
    Parameters:
        model_names: List of model names to compare
        train_df, val_df, test_df: DataFrames with 'sentence1', 'sentence2', 'label' columns
        model_config_kwargs: Dict with optional 'pretrained' and/or 'custom' keys that provide kwargs for the specific model types
        train_kwargs: Dict with optional 'pretrained' and/or 'custom' keys that provide kwargs for the specific model types
    Returns:
        DataFrame with comparison results sorted by F1 score
    """
    results = []
    
    for i, model_name in enumerate(model_names):
        print(f"\n{'#' * 70}")
        print(f"# Model {i+1}/{len(model_names)}: {model_name}")
        print(f"{'#' * 70}\n")
        
        try:
            is_custom = model_name.endswith(CUSTOM_MODEL_NAME)
            model_type = 'custom' if is_custom else 'pretrained'
            
            config_kwargs = {}
            if model_config_kwargs:
                config_kwargs = model_config_kwargs.get(model_type, model_config_kwargs.get('default', {}))
            
            config = get_model_config(model_name, device, **config_kwargs)
            
            # Try to load existing model or train new one
            try:
                if (not ALWAYS_TRAIN) and os.path.exists(config.model_path):
                    print(f"\nLoading trained model from {config.model_path}...")
                    model = load_model_from_config(config)
                else:
                    raise FileNotFoundError
            except Exception as e:
                if isinstance(e, FileNotFoundError):
                    print(f"Model not found at {config.model_path}.")
                else:
                    print(f"Could not load model: {e}")

                print(f"\nTraining model {model_name} from scratch...")

                current_train_kwargs = {}
                if train_kwargs:
                    current_train_kwargs = train_kwargs.get(model_type, train_kwargs.get('default', {}))
                
                model, _ = train_model(
                    config=config,
                    train_df=train_df,
                    val_df=val_df,
                    **current_train_kwargs
                )
            
            # Evaluate on test set
            test_dataset = config.dataset_class(test_df, config.tokenizer, config.max_length)
            test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=NUM_WORKERS)
            test_metrics = evaluate(model, test_loader, config.device)
            
            results.append({
                'model': model_name,
                'test_accuracy': test_metrics['accuracy'],
                'test_precision': test_metrics['precision'],
                'test_recall': test_metrics['recall'],
                'test_f1': test_metrics['f1'],
                'test_auc_roc': test_metrics['auc_roc'],
                'model_path': config.model_path,
                'status': 'success'
            })
            
        except Exception as e:
            print(f"ERROR training {model_name}: {e}")
            results.append({
                'model': model_name,
                'status': 'failed',
                'error': str(e)
            })
    
    return pd.DataFrame(results)

## 3.4 Prediction Function

A helper function to make predictions on individual sentence pairs.

In [ ]:
def predict_authorship_change(
    sentence1: str,
    sentence2: str,
    model: AuthorshipModel,
    tokenizer: AutoTokenizer,
    device: torch.device,
    max_length: int
) -> tuple[bool, float]:
    """
    Predict whether two sentences were written by different authors.
    
    Returns:
        (prediction, confidence): True if different authors, with confidence score
    """
    model.eval()
    
    if model.siamese:
        enc1 = tokenizer(sentence1, padding='max_length', truncation=True,
                        max_length=max_length, return_tensors='pt')
        enc2 = tokenizer(sentence2, padding='max_length', truncation=True,
                        max_length=max_length, return_tensors='pt')
        
        with torch.no_grad():
            outputs = model(
                enc1['input_ids'].to(device), enc1['attention_mask'].to(device),
                enc2['input_ids'].to(device), enc2['attention_mask'].to(device)
            )
    else:
        enc = tokenizer(sentence1, sentence2, padding='max_length', truncation=True,
                       max_length=max_length, return_tensors='pt')
        
        with torch.no_grad():
            outputs = model(enc['input_ids'].to(device), enc['attention_mask'].to(device))
    
    probs = F.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probs, dim=-1).item()
    confidence = probs[0][prediction].item()
    
    return bool(prediction), confidence

## 3.5 Data Splitting

We split the data into train (70%), validation (15%), and test (15%) sets with stratification to maintain class distribution.

In [ ]:
TRAIN_RATIO = 0.7
VALIDATION_TEST_RATIO = 0.5

train_df, temp_df = train_test_split(
    df, train_size=TRAIN_RATIO, random_state=RANDOM_SEED, stratify=df['label']
)
validation_df, test_df = train_test_split(
    temp_df, train_size=VALIDATION_TEST_RATIO, random_state=RANDOM_SEED, stratify=temp_df['label']
)

print("Dataset Splits:")
print(f"  Training:   {len(train_df):,} samples ({train_df['label'].mean():.2%} positive)")
print(f"  Validation: {len(validation_df):,} samples ({validation_df['label'].mean():.2%} positive)")
print(f"  Test:       {len(test_df):,} samples ({test_df['label'].mean():.2%} positive)")

---
# Part 4: Experiments

## 4.1 Train a Single Model

Let's train our custom lightweight transformer model and visualize the training progress. To change the model, you would just need to change the `model_name` variable with any of the names in the `get_model_config` function description.

In [ ]:
# f'{prefix}{name}' where prefix can be nothing or SIAMESE_PREFIX and name can be CUSTOM_MODEL_NAME , 'prajjwal1/bert-mini', etc.
model_name = CUSTOM_MODEL_NAME

config = get_model_config(model_name, device)

# Load existing model or train new one
try:
    if (not ALWAYS_TRAIN) and os.path.exists(config.model_path):
        print(f"\nLoading trained model from {config.model_path}...")
        model = load_model_from_config(config)
        result = None
    else:
        raise FileNotFoundError
except Exception as e:
    if isinstance(e, FileNotFoundError):
        print(f"Model not found at {config.model_path}")
    else:
        print(f"Could not load model: {e}")
    print(f"\nTraining model {model_name} from scratch...")
    model, result = train_model(
        config=config,
        train_df=train_df,
        val_df=validation_df
    )

In [ ]:
# Visualize training history if we just trained
if result is not None:
    history_df = pd.DataFrame(result.training_history)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss plot
    axes[0].plot(history_df['epoch'], history_df['train_loss'], 'b-o', label='Train Loss')
    axes[0].plot(history_df['epoch'], history_df['val_loss'], 'r-o', label='Validation Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Metrics plot
    axes[1].plot(history_df['epoch'], history_df['val_accuracy'], 'g-o', label='Accuracy')
    axes[1].plot(history_df['epoch'], history_df['val_f1'], 'm-o', label='F1 Score')
    axes[1].plot(history_df['epoch'], history_df['val_precision'], 'c-o', label='Precision')
    axes[1].plot(history_df['epoch'], history_df['val_recall'], 'y-o', label='Recall')
    axes[1].axvline(x=result.best_epoch, color='red', linestyle='--', alpha=0.7, label=f'Best Epoch ({result.best_epoch})')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Score')
    axes[1].set_title('Validation Metrics')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nBest F1: {result.best_f1:.4f} at epoch {result.best_epoch}")

## 4.2 Evaluate on Test Set

In [ ]:
print("\nEvaluating model on test set...")
test_dataset = config.dataset_class(test_df, config.tokenizer, config.max_length)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=NUM_WORKERS)
model = load_model_from_config(config)
test_metrics = evaluate(model, test_loader, device)

print("\n" + "=" * 60)
print("TEST SET RESULTS")
print("=" * 60)
print(f"\nModel: {config.model_name}")
print(f"\nMetrics:")
for key in ['accuracy', 'precision', 'recall', 'f1', 'auc_roc']:
    print(f"  {key.replace('_', ' ').title():12s}: {test_metrics[key]:.4f}")

# Part 5: Comparing Multiple Models

Let's compare different model architectures to find the best approach for this task.

In [ ]:
MODELS_TO_COMPARE = [
    CUSTOM_MODEL_NAME,                          # Custom lightweight transformer
    f'{SIAMESE_PREFIX}{CUSTOM_MODEL_NAME}',     # Siamese custom transformer
    'prajjwal1/bert-mini',                      # Pretrained BERT-mini
    f'{SIAMESE_PREFIX}prajjwal1/bert-mini',     # Siamese BERT-mini
    'microsoft/deberta-v3-small',               # (used by SCL-DeBERTa)
    f'{SIAMESE_PREFIX}microsoft/deberta-v3-small',
    'roberta-base',                             # (used by cornell-1 and better_call_claude)
    f'{SIAMESE_PREFIX}roberta-base'
]

COMPARISON_FILE_PATH = 'model_comparison_results.csv'

if os.path.exists(COMPARISON_FILE_PATH) and (not ALWAYS_TRAIN):
    comparison_df = pd.read_csv(COMPARISON_FILE_PATH)
    print(f"\nLoaded existing comparison from {COMPARISON_FILE_PATH}")
else:
    comparison_df = compare_models(
        model_names=MODELS_TO_COMPARE,
        train_df=train_df,
        val_df=validation_df,
        test_df=test_df,
        train_kwargs={'pretrained': {'freeze_encoder': 0.2}}
    )

    comparison_df.to_csv(COMPARISON_FILE_PATH, index=False)
    print(f"\nComparison saved to {COMPARISON_FILE_PATH}")

print("\n" + "=" * 80)
print("COMPARISON SUMMARY")
print("=" * 80)

successful = comparison_df[comparison_df['status'] == 'success']
if len(successful) > 0:
    print("\nTest Set Results (sorted by F1):")
    display_cols = ['model', 'test_f1', 'test_accuracy', 'test_precision', 'test_recall', 'test_auc_roc']
    print(successful[display_cols].sort_values('test_f1', ascending=False).to_string(index=False))
    
    best_model = successful.loc[successful['test_f1'].idxmax()]
    print(f"\nBest model: {best_model['model']} (Test F1: {best_model['test_f1']:.4f})")

failed = comparison_df[comparison_df['status'] == 'failed']
if len(failed) > 0:
    print(f"\nFailed models: {', '.join(failed['model'].tolist())}")

---
# Part 6: Inference Examples

## 6.1 Test on Dataset Samples

Let's test the model on some examples from the test set.

In [ ]:
# Load model and tokenizer for inference
model_name = CUSTOM_MODEL_NAME
config = get_model_config(model_name, device)
model = load_model_from_config(config)
tokenizer = AutoTokenizer.from_pretrained(config.model_path)

print(f"Model loaded: {model_name}")

In [ ]:
# Test on a few examples from the test set
print("\n" + "=" * 70)
print("EXAMPLE PREDICTIONS FROM TEST SET")
print("=" * 70)

num_examples = 5
correct = 0

for i in range(num_examples):
    row = test_df.iloc[i]
    test_sent1 = row['sentence1']
    test_sent2 = row['sentence2']
    true_label = row['label']
    
    pred, conf = predict_authorship_change(test_sent1, test_sent2, model, tokenizer, device, config.max_length)
    
    is_correct = pred == true_label
    correct += int(is_correct)
    
    print(f"\nExample {i+1}:")
    print(f"  Sentence 1: {test_sent1[:80]}..." if len(test_sent1) > 80 else f"  Sentence 1: {test_sent1}")
    print(f"  Sentence 2: {test_sent2[:80]}..." if len(test_sent2) > 80 else f"  Sentence 2: {test_sent2}")
    print(f"  True label: {'Different authors' if true_label else 'Same author'}")
    print(f"  Predicted:  {'Different authors' if pred else 'Same author'} (confidence: {conf:.2%})")
    print(f"  Result:     {'Correct' if is_correct else 'Incorrect'}")

print(f"\nAccuracy on these examples: {correct}/{num_examples} ({correct/num_examples:.0%})")

## 6.2 Test on Custom Sentences

You can try any model with by changing the sentence pairs.

In [ ]:
# Example 1: Same author (similar formal writing style)
sent1 = "The implementation of machine learning algorithms requires careful consideration of various hyperparameters."
sent2 = "Furthermore, the selection of appropriate evaluation metrics is crucial for assessing model performance."

pred, conf = predict_authorship_change(sent1, sent2, model, tokenizer, device, config.max_length)

print("Example 1 - Expected: Same author (formal academic style)")
print(f"  Sentence 1: {sent1}")
print(f"  Sentence 2: {sent2}")
print(f"  Prediction: {'Different authors' if pred else 'Same author'} (confidence: {conf:.2%})")
print()

In [ ]:
# Example 2: Different authors (contrasting styles)
sent1 = "The empirical analysis demonstrates a statistically significant correlation between the variables under investigation."
sent2 = "lol yeah that's pretty cool i guess, dunno why anyone would care tho tbh"

pred, conf = predict_authorship_change(sent1, sent2, model, tokenizer, device, config.max_length)

print("Example 2 - Expected: Different authors (formal vs. casual)")
print(f"  Sentence 1: {sent1}")
print(f"  Sentence 2: {sent2}")
print(f"  Prediction: {'Different authors' if pred else 'Same author'} (confidence: {conf:.2%})")
print()

---
## Part 7: Comparison with Other Publications

Now that we have developed and evaluated our models, we compare our best performing approach against state-of-the-art solutions from the PAN@CLEF 2025 Multi-Author Writing Style Analysis shared task. This comparison will help us understand how our best model performs relative to other approaches used by other research teams.

In [ ]:
best_model_name = "TODO"
f1_easy = 0.000    # TODO
f1_medium = 0.000  # TODO
f1_hard = 0.000    # TODO

sota_results = [
    {"Team": "better_call_claude", "Approach": "SSPC (BiLSTM/PLM)", "Easy": 0.929, "Medium": 0.815, "Hard": 0.731},
    {"Team": "stylospies", "Approach": "Graph/Structural Features (SBERT/GB)", "Easy": 0.959, "Medium": 0.786, "Hard": 0.791},
    {"Team": "TMU", "Approach": "Ensemble LaBSE/Siamese BiLSTM", "Easy": 0.950, "Medium": 0.792, "Hard": 0.792},
    {"Team": "cornell-1", "Approach": "Ensembled-BertStyleNN", "Easy": 0.909, "Medium": 0.793, "Hard": 0.698},
    {"Team": "OpenFact", "Approach": "Punctuation-Guided Pretraining", "Easy": 0.919, "Medium": 0.771, "Hard": 0.752},
    {"Team": "xxsu-team", "Approach": "SCL-DeBERTa (SCL)", "Easy": 0.955, "Medium": 0.825, "Hard": 0.829},
    # Arturo + Radu: Arduo sounds like Arturo and contains all leters in Radu. It also means hard work, and since it ends with 'duo' it hints at collaboration.
    {"Team": "Arduo", "Approach": f"{best_model_name}", "Easy": f1_easy, "Medium": f1_medium, "Hard": f1_hard}
]

sota_df = pd.DataFrame(sota_results)
sota_df['Average F1'] = sota_df[['Easy', 'Medium', 'Hard']].mean(axis=1)

sota_df_sorted = sota_df.sort_values('Average F1', ascending=False).round(3)

print("\n" + "=" * 100)
print("COMPARISON WITH PUBLISHED PAN 2025 RESULTS")
print("Multi-Author Writing Style Analysis - F1 Macro Scores")
print("=" * 100)
display(sota_df_sorted.style.hide(axis='index').format(precision=2))

---
# Conclusion

## Summary

In this project, we built and evaluated several transformer-based models for multi-author writing style analysis. Key findings:

1. **Class Imbalance**: The dataset is heavily imbalanced. TODO: write more about this

2. **Model Architectures**: We compared custom lightweight transformers with pretrained BERT models in both standard and siamese configurations.

3. **Regularization**: Label smoothing and dropout helped reduce overfitting, which was a significant challenge.

## Development Process

### Complete Development History

The full evolution of this project, including all experiments, failed attempts, and incremental improvements, is preserved in our [public GitHub repository commit history](https://github.com/RaduB20/multi_author_writing_style_analysis). Below, we explain the key decisions and their rationale so you don't need to dig through commits:

### Development Timeline

Our workflow and the implementations we did followed this progression:

1. **Initial Data Pipeline**
   - Basic `load_data()` function without augmentation
   - Simple train/val/test splits

2. **Pretrained Models Integration**
   - HuggingFace transformer integration
   - Training and evaluation pipeline for BERT-based models
   - **Problem**: Training was slow (~X minutes per epoch) due to large parameter count
   - **Solution**: Needed faster iteration cycles for debugging techniques and experimentation

3. **Custom Architecture Development**
   - Designed `LightweightTransformerEncoder` for rapid prototyping
   - **Problem**: Code duplication between custom and pretrained pipelines
   - **Solution**: Created `AuthorshipModel` class with HuggingFace-compatible interface for both types of models

4. **Addressing Overfitting**
   - **Critical observation**: Train accuracy ~90%, test accuracy ~50% (massive overfitting)
   - Increased dropout from 0.1 -> 0.33
   - Implemented label smoothing (0.1)
   - Added gradient clipping (max_norm=1.0)
   - Added weighted sampling for class imbalance
   - Implemented data augmentation in `load_data()`

5. **Siamese Architecture**
   - **Motivation**: After establishing a solid baseline, we hypothesized that encoding sentences separately and comparing their representations might better capture stylistic differences
   - Implemented multiple similarity methods: concatenation, difference, multiplication, cosine
   - Flexible architecture allowing any combination of similarity features

6. **Evaluation**
   - `compare_models()` function for systematic model comparison
   - Prediction interface for inference on new sentence pairs
   - Training history visualization

### On the Absence of Intermediate Versions

This notebook contains only the working version because:
- We never removed functionality, only added it
- Each feature builds on the previous foundation
- Showing intermediate versions would be redundant (they are subsets of the final code)

## Comparison to State-of-the-Art

TODO: Research and compare our approach to current SOTA methods for authorship verification and multi-author writing style analysis.

## Future Improvements

If we were to continue improving this project, some possible directions include:

- **Apply Gridsearch**:
- **Data Augmentation**: Using techniques like back-translation or paraphrasing to create more diverse training examples. Even SMOTE could be helpful when doing data augmentation from the vectorized sentences.
- **Focal Loss**: A loss function specifically designed for imbalanced datasets that down-weights easy examples
- **Larger Pretrained Models**: Fine-tuning larger models like DeBERTa or RoBERTa for better performance

---
## References

- PAN25 Multi-Author Analysis Task: [https://pan.webis.de/clef25/pan25-web/style-change-detection.html](https://pan.webis.de/clef25/pan25-web/style-change-detection.html)
- Dataset: [https://zenodo.org/records/14891299](https://zenodo.org/records/14891299)
- HuggingFace Transformers: [https://huggingface.co/transformers/](https://huggingface.co/transformers/)
- MTEB Leaderboard: [https://huggingface.co/spaces/mteb/leaderboard](https://huggingface.co/spaces/mteb/leaderboard)


### Publicated Models References

The external comparison results are based on the following publications from the PAN Lab at CLEF 2025 (provided in BibTeX format for easy citation):

*   **Team stylospies**:
    ```bibtex
    @article{boriceanu2025style,
      title={Style change detection using graph and structural-linguistic features for multi-author writing analysis},
      author={Boriceanu, I and B{\u{a}}ltoiu, A},
      journal={Working Notes of CLEF},
      year={2025}
    }
    ```
*   **Team better\_call\_claude**:
    ```bibtex
    @article{schmidt2025team,
      title={Team" better\_call\_claude": Style Change Detection using a Sequential Sentence Pair Classifier},
      author={Schmidt, Gleb and R{\"o}misch, Johannes and Halchynska, Mariia and Gorovaia, Svetlana and Yamshchikov, Ivan P},
      journal={arXiv preprint arXiv:2508.00675},
      year={2025}
    }
    ```
*   **Team OpenFact**:
    ```bibtex
    @article{ksikezniak2025openfact,
      title={OpenFact at PAN 2025: punctuation-guided pretraining for sentence-level style change detection},
      author={Ksi{\k{e}}{\.z}niak, Ewelina and W{\k{e}}cel, Krzysztof and Sawi{\'n}ski, Marcin},
      journal={Working Notes of CLEF},
      year={2025}
    }
    ```
*   **Team xxsu-team**:
    ```bibtex
    @article{lin2025scl,
      title={SCL-DeBERTa: multi-author writing style change detection enhanced by supervised contrastive learning},
      author={Lin, Kaichuan and Liu, Chang and Ye, Fuchuan and Han, Zhongyuan},
      journal={Working Notes of CLEF},
      year={2025}
    }
    ```
*   **Team TMU**:
    ```bibtex
    @article{hosseinbeigi2025team,
      title={Team TMU at PAN 2025: an ensemble of fine-tuned LaBSE and siamese neural network for multi-author writing style analysis},
      author={Hosseinbeigi, Sara Bourbour and Mehrani, Ali},
      journal={Working Notes of CLEF},
      year={2025}
    }
    ```
*   **Team cornell-1**:
    ```bibtex
    @article{boloni2025team,
      title={Team cornell-1 at PAN: ensembling fine-tuned transformer models for writing style analysis},
      author={Boloni-Turgut, D and Verma, Dhriti and Cardie, Claire},
      journal={Working Notes of CLEF},
      year={2025}
    }
    ```